In [29]:
# -*- coding: utf-8 -*-

import re
import pywt
import math
import datetime
import numpy as np
import fastcluster
import pandas as pd


from text_processor import MDB, text_process
from utils import load_obj, dump_obj, progress
from timeseries_utils import *
from metrics import *
from pyculiarity import detect_ts
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import cosine
from geopy.distance import vincenty
from gensim.models import Word2Vec
from scipy.stats import pearsonr, spearmanr
from meta import geodata
from datetime import datetime

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [30]:
md = MDB('tweets')
city = 'Miami'
mxscl = 5

connected to database


In [31]:
cur = md.get_last(city, days=0, hours=4)
prc, st, en = process_batch(cur, geo=True, fsw=True, stem=True)

found 4005 documents
querying took 0:00:01.176305
100%

retrieval and processing took 0:00:01.907505


In [ ]:
f = [t['words'] for t in prc if len(list_intersection(t['words'],b1))>1]

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("4 Pennsilvanya")
print(location.address)

In [ ]:
md.client.local.collection_names()

In [32]:
prc[10:100]

[{'created_at': datetime.datetime(2016, 4, 1, 22, 3, 51),
  'geo': (0, [-83.804474999999996, 27.698681999999998]),
  'words': [u'bore', u'as', u'all', u'hell']},
 {'created_at': datetime.datetime(2016, 4, 1, 22, 3, 54),
  'geo': (0, [-80.428181999999993, 25.610864499999998]),
  'words': [u'when',
   u'you',
   u'tri',
   u'click',
   u'on',
   u'your',
   u'boy',
   u'caus',
   u'you',
   u'wanna',
   u'get',
   u'your',
   u'shit',
   u'done',
   u'and',
   u'they',
   u'say',
   u'il',
   u'take',
   u'you',
   u'dumbi',
   u'lol',
   u'#squad']},
 {'created_at': datetime.datetime(2016, 4, 1, 22, 3, 54),
  'geo': (0, [-80.233328499999999, 25.782353499999999]),
  'words': [u'friday',
   u'friday',
   u'friiiday',
   u'what',
   u'to',
   u'do',
   u'what',
   u'to',
   u'do',
   u'what',
   u'to',
   u'doooo']},
 {'created_at': datetime.datetime(2016, 4, 1, 22, 3, 55),
  'geo': (0, [-83.804474999999996, 27.698681999999998]),
  'words': [u'scortworkz',
   u'froggindirti',
   u'ohhhh',


In [11]:
lm = WordNetLemmatizer()

In [21]:
prc[0]['words']

['call',
 'un',
 'american',
 'one',
 'koolaid',
 'drinker',
 'fact',
 'i',
 'm',
 'welfare',
 'never',
 'never']

In [22]:
lm.lemmatize(prc[0]['words'][10])

'never'

In [10]:
from nltk.stem import WordNetLemmatizer

In [ ]:
bbox = [g['crds'] for g in geodata if g['pDesc'] == city][0]

g = []
F = []
res = 30
MTR = cell_matrix(res)

for i, p in enumerate(prc):
    if p['geo'][0] == 1:
        g.append(p)

for o in g:
    cell = assign_cell(o, bbox, res)
    if cell:
        cx, cy = cell
        F.append(o)
        MTR[cx][cy]['objects'].append(o)

for i in range(res):
    for j in range(res):
        progress(i*res+j, res*res, skip=1)
        r = build_timeseries(MTR[i][j]['objects'], st, en, mins = 15, filtration=1, logging=False)
        if r: 
            M, voc = r
        else:
            M, voc = None, None
        MTR[i][j]['series'] = M
        MTR[i][j]['voc'] = voc

In [ ]:
def w2v_represent(model, words):
    WW = []
    for word in words:
        try:
            T = model[word]
            if T.shape[0]:
                WW.append(T)
        except Exception as e:
            pass
                
    if len(WW):
        return np.array(WW)
    else:
        return None

def w2v_similarity_1(model, stra, strb):
    sm = w2v_represent(model, stra), w2v_represent(model, strb)
    return similarity(sm[0], sm[1])/2

def w2v_similarity_2(model, stra, strb):
    w1 = w2v_represent(model, stra), w2v_represent(model, strb)
    w2 = sim_matrix(w1[0], w1[1])
    w3 = do_align(w2).astype(int)
    w31, w32 = w3.nonzero()
    w4 = np.mean([w2[z[0]][z[1]] for z in zip(w31, w32)])
    return w4

In [ ]:
w2 = w2v_similarity_2(w2v, lr[5][0]['words'], lr[5][1]['words'])

In [ ]:
def sim_matrix(v1,v2):
    if v1.shape[0] and v2.shape[0] and v1.shape[1] == v2.shape[1]:
        sim = 1 - cdist(v1, v2, 'cosine')
        return sim
    else:
        return 0

def similarity(v1,v2):
    sim = sim_matrix(v1,v2)
    thresh = 0.5
    sk = (sim[sim > thresh])
    sk /= sk.shape[0]
    if type(sim) == int : 
        return 0
    else:
        return sum(sk)
    
def do_align(MT):
    thresh = 0.4
    align = np.zeros(MT.shape)
    if MT.shape[0] <= MT.shape[1]:
        maxs = np.argmax(MT,0)        
        for i,m in enumerate(maxs):
            if MT[maxs[i],i] >= thresh:
                align[maxs[i],i] = 1
    if MT.shape[0] > MT.shape[1]:
        maxs = np.argmax(MT,1)
        for i,m in enumerate(maxs):
            if MT[i,maxs[i]] >= thresh:
                align[i,maxs[i]] = 1
    return align

def sim_align(v1,v2):
    sim = sim_matrix(v1,v2)
    return sum(sim[allign(sim).astype(int)])

In [ ]:
def eval_correlations(cr):
    C = [[], []]
    for k in cr.keys():
        for i,c in enumerate(cr[k].values()):
            C[i].append(c[0])
    return np.array(C), cr.keys()

In [ ]:
lr = []
for z1, i in enumerate(F):
    for z2, j in enumerate(F):
        if len(list_intersection(i['words'], j['words'])) > 3 and i['words']!=j['words'] and z1!=z2:
            lr.append((i,j))
            if len(lr) > 1000: break

In [ ]:
def wavesim(u,v):
    cors = {}
    
    for intr in list_intersection(u['words'], v['words']):
        try:
            scl = mxscl - spatialscale(eu_distance(u['geo'][1], v['geo'][1]), 0, mxrang)

            c1x, c1y = assign_cell(u, bbox, res)
            id1 = MTR[c1x][c1y]['voc'][intr]
            ser1 = MTR[c1x][c1y]['series'][id1]
            dw1 = DWTransform(ser1, scl)


            c2x, c2y = assign_cell(v, bbox, res)
            id2 = MTR[c2x][c2y]['voc'][intr]
            ser2 = MTR[c2x][c2y]['series'][id2]
            dw2 = DWTransform(ser2, scl)
            
            ss = [np.concatenate(dw1),np.concatenate(dw2)]
            cors[intr]= {'pearson': pearsonr(ss[0], ss[1]), 'spearman': spearmanr(ss[0], ss[1])}
        except Exception as e:
            pass
    
    return cors

In [ ]:
def evalpair(a,b):
    c, keys = eval_correlations(wavesim(a, b))
    w1 = w2v_similarity_1(w2v, a, b)
    w2 = w2v_similarity_2(w2v, a, b)
    t1 = tf_similarity(TF, a, b)

    A = np.array([keys, w1*c ,w2*c, t1*c])
    return A

In [ ]:
class Lines(object):
    def __init__(self, input):
        self.filenames = np.atleast_1d(input)

    def __iter__(self):
        for filename in self.filenames:
            try:
                for line in open(filename, 'r'):
                    try:
                        yield line
                    except:
                        yield []
            except:
                yield[]

    def count_samples(self):
        cnt = 0
        for filename in self.filenames:
            for line in open(filename, 'r'):
                cnt += 1
        return cnt

In [ ]:
lins = Lines(['/ssd/raw_text_datasets/'+f for f in os.listdir('/ssd/raw_text_datasets')])

In [ ]:
TF = TfidfVectorizer('content', decode_error='ignore', stop_words=sw, token_pattern=tok_regex, min_df=10)

In [ ]:
def tf_similarity(model, a, b):
        return 1 - cosine(model.transform([' '.join(a['words'])]).toarray(),model.transform([' '.join(b['words'])]).toarray())

def spatialscale(d, minrang, maxrang, st = mxscl):
    lspace = np.logspace(minrang, maxrang, st, base=np.e)
    r = np.e**d
    for i, s in enumerate(lspace):
        if s > r: break
    return i

def eu_distance(u, v):
    return vincenty(u[::-1], v[::-1]).kilometers

def pearsoncora(u,v):
    return pearsonr(u,v)[0]

def pearsoncorb(u,v):
    return pearsonr(u,v)[1]

def DWTransform(ts, level, plt = False):
    ca = ts
    for l in range(level):
        ca, cd = pywt.dwt(ca, 'haar', mode='zero')
    if plt:
        plot(ca, 0)
        plot(cd, 0)
    return ca, cd

mxrang = eu_distance(bbox[0:2], bbox[2:4])
mnrang = min((bbox[3]-bbox[1])/res, (bbox[2]-bbox[0])/res)
mnrang = 0

In [ ]:
tf_similarity(TF, lr[0][0], lr[0][1])

In [ ]:
import gensim, os

In [ ]:
w2v = gensim.models.Word2Vec.load('assets/w2v/w2v_model_v3')

In [ ]:
w2vt = gensim.models.Word2Vec.load('assets/w2v/w2v_model_v1')

In [ ]:
z = MC.shape[0]
pdist = np.zeros((z, z))

for i in range(z):
    for j in range(z):
        if not pdist[i][j]:
            progress(i*z + j, z*z, skip = 1000)
            r = np.array(pearsonr(MC[i], MC[j])[1])
            pdist[i][j] = r
            pdist[j][i] = r

In [ ]:
dump_obj(TF, 'assets/TF_VECTORISER')

In [ ]:
def show(M, n, l = 1):
    print n, voc[n], '----------------------------------->>'
    plot(M[n], 1)
    
    print 'DWT LEVEL', l ,'---------------------------------->>'
    da, dd = DWTransform(M[n], l, plt = True)    

In [ ]:
sw = load_stop_words()

In [ ]:
pb = pairwise_distances(MC, metric=pearsoncorb, n_jobs=8)

In [ ]:
import nltk
from stopwords import stopwords
def load_stop_words():
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words.extend(stopwords)
    return set(stop_words)

In [ ]:
def w2v_transform(W, model, dim = 256):
    st = datetime.now()
    l = len(W)
    M = []
    for i, sent in enumerate(W):
        words = sent
        b = []
        for word in words:
            try:
                b.append(model[word])
            except:
                pass
            
        B = np.array(b)
        if not isinstance(B, np.ndarray):
            B = np.zeros((1, dim))

        M.append(B)
        progress(i, l)
        
    M = np.array(M)
    F = []
    idx = []
    for i, m in enumerate(M):
        if m.shape[0]:
            F.append(m)
            idx.append(i)
        
    
    print '\ntransforming took', datetime.now() - st
    return np.array(F), idx

In [ ]:
def pr_dist(M):
    dim = M.shape[0]
    R = np.zeros((dim,dim))
    c = 0
    for i in range(dim):
        for j in range(dim):
            progress(i*dim + j, dim*dim, skip = 100)
            if i == j:
                R[i][j] = 0
            elif not R[i][j] and not R[j][i]:
                r = DTWDistance(M[i], M[j])
                c += 1
                R[i][j] = r
                R[j][i] = r
    return R, c

In [ ]:
st = datetime.now()
s_sim = pairwise_distances(P, metric=DTW_hybrid, n_jobs=12)
en = datetime.now()
print str(en-st)

In [ ]:
windows = np.array(range(P.shape[1]))
mv_wind_size = int(0.05*len(windows))

for term_n in range(P.shape[0]):
    print 'term: >>>', voc[ind[term_n]]
    plt.plot(windows, P[term_n], 'b-', label='label here')
    plt.plot(windows, movingaverage(P[term_n], mv_wind_size), 'r-', label='label here')
    plt.show()

In [ ]:
def estimated_autocorrelation(x):
    n = len(x)
    variance = x.var()
    x = x-x.mean()
    r = np.correlate(x, x, mode = 'full')[-n:]
    assert np.allclose(r, np.array([(x[:n-k]*x[-(n-k):]).sum() for k in range(n)]))
    result = r/(variance*(np.arange(n, 0, -1)))
    return result

In [ ]:
react = 'отлично, замечательно'